# Machine Learning
Cassady Jackson and Baylie Schnieder

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, r2_score, f1_score
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy.stats import chi2_contingency
from scipy.stats import pearsonr

In [3]:
state_data = pd.read_csv('State_Unique_Data.csv')

In [4]:
state_data

,YearEnd,LocationAbbr,State,DataSource,Topic,Question,DataValueUnit,DataValueType,DataValue,DataValueAlt,...,Stratification1,LocationID,TopicID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1,Overall Public School Rank 2021,Higher Ed Quality 2021,School Safety Rank 2021
0,2019,AK,Alaska,NVSS,Cardiovascular Disease,Diseases of the heart mortality among all peop...,"cases per 100,000",Age-adjusted Rate,159.70,159.70,...,Male,2,CVD,CVD09,AGEADJRATE,SEX,SEXM,48,50,34
1,2019,CA,California,BRFSS,Cardiovascular Disease,Taking medicine to control high blood pressure...,%,Age-adjusted Prevalence,47.20,47.20,...,"Multiracial, non-Hispanic",6,CVD,CVD02,AGEADJPREV,RACE,MRC,37,38,32
2,2019,AL,Alabama,CMS Part A Claims Data,Cardiovascular Disease,Hospitalization for heart failure as principal...,"cases per 1,000",Crude Rate,71.43,71.43,...,"American Indian or Alaska Native, non-Hispanic",1,CVD,CVD06,CRDRATE,RACE,AIAN,44,46,17
3,2019,CA,California,BRFSS,Cardiovascular Disease,Taking medicine for high cholesterol among adults,%,Crude Prevalence,31.60,31.60,...,Male,6,CVD,CVD04,CRDPREV,SEX,SEXM,37,38,32
4,2019,AZ,Arizona,NVSS,Cardiovascular Disease,Cerebrovascular disease (stroke) mortality amo...,"cases per 100,000",Crude Rate,26.10,26.10,...,"Black, non-Hispanic",4,CVD,CVD07,CRDRATE,RACE,BLK,49,50,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16675,2021,WY,Wyoming,BRFSS,Cardiovascular Disease,High cholesterol among adults who have been sc...,%,Crude Prevalence,32.80,32.80,...,Age 45-64,56,CVD,CVD03,CRDPREV,AGE,AGE4564,19,18,29
16676,2021,WY,Wyoming,BRFSS,Cardiovascular Disease,High blood pressure among adults,%,Age-adjusted Prevalence,39.90,39.90,...,Hispanic,56,CVD,CVD01,AGEADJPREV,RACE,HIS,19,18,29
16677,2021,WY,Wyoming,BRFSS,Cardiovascular Disease,High blood pressure among adults,%,Crude Prevalence,32.20,32.20,...,"Multiracial, non-Hispanic",56,CVD,CVD01,CRDPREV,RACE,MRC,19,18,29
16678,2021,WY,Wyoming,NVSS,Cardiovascular Disease,Coronary heart disease mortality among all peo...,"cases per 100,000",Crude Rate,50.70,50.70,...,Hispanic,56,CVD,CVD08,CRDRATE,RACE,HIS,19,18,29


In [11]:
dummies2 = pd.get_dummies(state_data['Topic'])

states_data = pd.concat([state_data, dummies2], axis=1)
states_data.drop(['StratificationID1','StratificationCategoryID1','DataValueTypeID','QuestionID', 'TopicID', 'StratificationCategory1', 'DataValueType', 'DataValueUnit', 'Question','DataSource', 'LocationID', 'Stratification1', 'LocationAbbr', 'State', 'Topic', 'YearEnd', 'Higher Ed Quality 2021', 'School Safety Rank 2021', 'DataValue', 'DataValueAlt', 'LowConfidenceLimit', 'HighConfidenceLimit' ], axis=1, inplace=True)
display(states_data)

,Overall Public School Rank 2021,Cardiovascular Disease
0,48,True
1,37,True
2,44,True
3,37,True
4,49,True
...,...,...
16675,19,True
16676,19,True
16677,19,True
16678,19,True


### Grouping by Top 10 State Rankings and Bottom 10 State Rankings

In [18]:
def top_ten(state):
    if state <= 10:
        return 'Top 10'
    elif 10 < state <= 41:
        return 'Middle'
    else:
        return 'Bottom 10'

state_data['Ranking Level'] = state_data['Overall Public School Rank 2021'].apply(top_ten)

display(state_data)

,YearEnd,LocationAbbr,State,DataSource,Topic,Question,DataValueUnit,DataValueType,DataValue,DataValueAlt,...,LocationID,TopicID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1,Overall Public School Rank 2021,Higher Ed Quality 2021,School Safety Rank 2021,Ranking Level
0,2019,AK,Alaska,NVSS,Cardiovascular Disease,Diseases of the heart mortality among all peop...,"cases per 100,000",Age-adjusted Rate,159.70,159.70,...,2,CVD,CVD09,AGEADJRATE,SEX,SEXM,48,50,34,Bottom 10
1,2019,CA,California,BRFSS,Cardiovascular Disease,Taking medicine to control high blood pressure...,%,Age-adjusted Prevalence,47.20,47.20,...,6,CVD,CVD02,AGEADJPREV,RACE,MRC,37,38,32,Middle
2,2019,AL,Alabama,CMS Part A Claims Data,Cardiovascular Disease,Hospitalization for heart failure as principal...,"cases per 1,000",Crude Rate,71.43,71.43,...,1,CVD,CVD06,CRDRATE,RACE,AIAN,44,46,17,Bottom 10
3,2019,CA,California,BRFSS,Cardiovascular Disease,Taking medicine for high cholesterol among adults,%,Crude Prevalence,31.60,31.60,...,6,CVD,CVD04,CRDPREV,SEX,SEXM,37,38,32,Middle
4,2019,AZ,Arizona,NVSS,Cardiovascular Disease,Cerebrovascular disease (stroke) mortality amo...,"cases per 100,000",Crude Rate,26.10,26.10,...,4,CVD,CVD07,CRDRATE,RACE,BLK,49,50,31,Bottom 10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16675,2021,WY,Wyoming,BRFSS,Cardiovascular Disease,High cholesterol among adults who have been sc...,%,Crude Prevalence,32.80,32.80,...,56,CVD,CVD03,CRDPREV,AGE,AGE4564,19,18,29,Middle
16676,2021,WY,Wyoming,BRFSS,Cardiovascular Disease,High blood pressure among adults,%,Age-adjusted Prevalence,39.90,39.90,...,56,CVD,CVD01,AGEADJPREV,RACE,HIS,19,18,29,Middle
16677,2021,WY,Wyoming,BRFSS,Cardiovascular Disease,High blood pressure among adults,%,Crude Prevalence,32.20,32.20,...,56,CVD,CVD01,CRDPREV,RACE,MRC,19,18,29,Middle
16678,2021,WY,Wyoming,NVSS,Cardiovascular Disease,Coronary heart disease mortality among all peo...,"cases per 100,000",Crude Rate,50.70,50.70,...,56,CVD,CVD08,CRDRATE,RACE,HIS,19,18,29,Middle


In [20]:
top_ten = state_data.copy()
bottom_ten = state_data.copy()

In [22]:
top_ten = top_ten.drop(top_ten[top_ten['Ranking Level'] != 'Top 10'].index)
display(top_ten)
top_ten.to_csv('TopTenStates.csv', header=True, index=False, encoding="utf-8")

,YearEnd,LocationAbbr,State,DataSource,Topic,Question,DataValueUnit,DataValueType,DataValue,DataValueAlt,...,LocationID,TopicID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1,Overall Public School Rank 2021,Higher Ed Quality 2021,School Safety Rank 2021,Ranking Level
710,2019,CT,Connecticut,BRFSS,Cardiovascular Disease,Taking medicine to control high blood pressure...,%,Crude Prevalence,81.20,81.20,...,9,CVD,CVD02,CRDPREV,RACE,WHT,2,2,19,Top 10
717,2019,DE,Delaware,NVSS,Cardiovascular Disease,Diseases of the heart mortality among all peop...,"cases per 100,000",Age-adjusted Rate,150.00,150.00,...,10,CVD,CVD09,AGEADJRATE,RACE,BLK,9,15,2,Top 10
719,2019,CT,Connecticut,BRFSS,Cardiovascular Disease,Taking medicine for high cholesterol among adults,%,Crude Prevalence,34.20,34.20,...,9,CVD,CVD04,CRDPREV,OVERALL,OVR,2,2,19,Top 10
721,2019,CT,Connecticut,BRFSS,Cardiovascular Disease,Taking medicine to control high blood pressure...,%,Crude Prevalence,77.70,77.70,...,9,CVD,CVD02,CRDPREV,OVERALL,OVR,2,2,19,Top 10
727,2019,DE,Delaware,CMS Part A Claims Data,Cardiovascular Disease,Hospitalization for heart failure as principal...,"cases per 1,000",Age-adjusted Rate,19.07,19.07,...,10,CVD,CVD06,AGEADJRATE,RACE,API,9,15,2,Top 10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16667,2021,WI,Wisconsin,BRFSS,Cardiovascular Disease,Taking medicine for high cholesterol among adults,%,Crude Prevalence,24.90,24.90,...,55,CVD,CVD04,CRDPREV,RACE,HIS,8,5,24,Top 10
16669,2021,WI,Wisconsin,CMS Part A Claims Data,Cardiovascular Disease,Hospitalization for heart failure as principal...,"cases per 1,000",Crude Rate,23.33,23.33,...,55,CVD,CVD06,CRDRATE,SEX,SEXM,8,5,24,Top 10
16670,2021,WI,Wisconsin,NVSS,Cardiovascular Disease,Diseases of the heart mortality among all peop...,"cases per 100,000",Crude Rate,18.20,18.20,...,55,CVD,CVD09,CRDRATE,RACE,MRC,8,5,24,Top 10
16672,2021,WI,Wisconsin,NVSS,Cardiovascular Disease,Cerebrovascular disease (stroke) mortality amo...,"cases per 100,000",Age-adjusted Rate,34.60,34.60,...,55,CVD,CVD07,AGEADJRATE,RACE,WHT,8,5,24,Top 10


In [24]:
bottom_ten = bottom_ten.drop(bottom_ten[bottom_ten['Ranking Level'] != 'Bottom 10'].index)
display(bottom_ten)
bottom_ten.to_csv('BottomTen.csv', header=True, index=False, encoding="utf-8")

,YearEnd,LocationAbbr,State,DataSource,Topic,Question,DataValueUnit,DataValueType,DataValue,DataValueAlt,...,LocationID,TopicID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1,Overall Public School Rank 2021,Higher Ed Quality 2021,School Safety Rank 2021,Ranking Level
0,2019,AK,Alaska,NVSS,Cardiovascular Disease,Diseases of the heart mortality among all peop...,"cases per 100,000",Age-adjusted Rate,159.70,159.70,...,2,CVD,CVD09,AGEADJRATE,SEX,SEXM,48,50,34,Bottom 10
2,2019,AL,Alabama,CMS Part A Claims Data,Cardiovascular Disease,Hospitalization for heart failure as principal...,"cases per 1,000",Crude Rate,71.43,71.43,...,1,CVD,CVD06,CRDRATE,RACE,AIAN,44,46,17,Bottom 10
4,2019,AZ,Arizona,NVSS,Cardiovascular Disease,Cerebrovascular disease (stroke) mortality amo...,"cases per 100,000",Crude Rate,26.10,26.10,...,4,CVD,CVD07,CRDRATE,RACE,BLK,49,50,31,Bottom 10
5,2019,AK,Alaska,NVSS,Cardiovascular Disease,Cerebrovascular disease (stroke) mortality amo...,"cases per 100,000",Crude Rate,20.20,20.20,...,2,CVD,CVD07,CRDRATE,AGE,AGE4564,48,50,34,Bottom 10
6,2019,AZ,Arizona,CMS Part A Claims Data,Cardiovascular Disease,Hospitalization for heart failure as principal...,"cases per 1,000",Age-adjusted Rate,43.89,43.89,...,4,CVD,CVD06,AGEADJRATE,RACE,BLK,49,50,31,Bottom 10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15448,2021,SC,South Carolina,BRFSS,Cardiovascular Disease,Taking medicine to control high blood pressure...,%,Age-adjusted Prevalence,66.90,66.90,...,45,CVD,CVD02,AGEADJPREV,RACE,BLK,43,42,41,Bottom 10
15464,2021,OK,Oklahoma,NVSS,Cardiovascular Disease,Coronary heart disease mortality among all peo...,"cases per 100,000",Crude Rate,131.50,131.50,...,40,CVD,CVD08,CRDRATE,OVERALL,OVR,47,48,20,Bottom 10
15467,2021,SC,South Carolina,BRFSS,Cardiovascular Disease,Taking medicine to control high blood pressure...,%,Crude Prevalence,79.40,79.40,...,45,CVD,CVD02,CRDPREV,SEX,SEXM,43,42,41,Bottom 10
15472,2021,OK,Oklahoma,BRFSS,Cardiovascular Disease,High blood pressure among adults,%,Crude Prevalence,22.70,22.70,...,40,CVD,CVD01,CRDPREV,RACE,HIS,47,48,20,Bottom 10


### Linear Regression

In [47]:
states_data.head()
X = states_data.drop('Overall Public School Rank 2021', axis=1)
y = states_data['Overall Public School Rank 2021'].values
X_train, X_test, y_train, y_test = train_test_split(X ,y, test_size = 0.2, random_state = 42)
Lin_Regression = LinearRegression().fit(X_train, y_train)
y_pre = Lin_Regression.predict(X_test)
Lin_Regression.score(X_test, y_test)

-7.102524701330104e-05

### Pearson Correlation

Used to understand the strength and direction of the linear relationship between two variables.

In [27]:
# calculate Pearson correlation between 'Bottom 10' Ranking and DataValueUnit (instances of chronic illness) 
pearson_corr = bottom_ten["Overall Public School Rank 2021"].corr(bottom_ten["DataValue"], method="pearson")

# display the result
print(f"Pearson Correlation: {pearson_corr:.4f}")

# interpretation
if abs(pearson_corr) < 0.3:
    print("Interpretation: No or weak linear correlation.")
elif 0.3 <= abs(pearson_corr) < 0.7:
    print("Interpretation: Moderate linear correlation.")
else:
    print("Interpretation: Strong linear correlation.")

Pearson Correlation: -0.0355
Interpretation: No or weak linear correlation.


Used to test the hypothesis found in the Pearson correlation and understand if it is significant or could have occured by chance

In [32]:
# perform the Pearson correlation test
corr_stat, p_value = pearsonr(bottom_ten["Overall Public School Rank 2021"], bottom_ten["DataValue"])

# display the correlation statistic and p-value
print(f"Pearson Correlation Coefficient: {corr_stat:.4f}")
print(f"P-value: {p_value:.4f}")

# interpretation based on p-value
alpha = 0.05  # significance level

if p_value > alpha:
    print("Fail to reject H0: No significant correlation (correlation = 0).")
else:
    print("Reject H0: Significant correlation exists (correlation != 0).")

Pearson Correlation Coefficient: -0.0355
P-value: 0.0376
Reject H0: Significant correlation exists (correlation != 0).


In [34]:
# calculate Pearson correlation between 'Bottom 10' Ranking and DataValueUnit (instances of chronic illness) 
top_pearson_corr = top_ten["Overall Public School Rank 2021"].corr(top_ten["DataValue"], method="pearson")

# display the result
print(f"Pearson Correlation: {top_pearson_corr:.4f}")

# interpretation
if abs(top_pearson_corr) < 0.3:
    print("Interpretation: No or weak linear correlation.")
elif 0.3 <= abs(top_pearson_corr) < 0.7:
    print("Interpretation: Moderate linear correlation.")
else:
    print("Interpretation: Strong linear correlation.")

Pearson Correlation: 0.0224
Interpretation: No or weak linear correlation.


In [36]:
# perform the Pearson correlation test
corr_stat, p_value = pearsonr(top_ten["Overall Public School Rank 2021"], top_ten["DataValue"])

# display the correlation statistic and p-value
print(f"Pearson Correlation Coefficient: {corr_stat:.4f}")
print(f"P-value: {p_value:.4f}")

# interpretation based on p-value
alpha = 0.05  # significance level

if p_value > alpha:
    print("Fail to reject H0: No significant correlation (correlation = 0).")
else:
    print("Reject H0: Significant correlation exists (correlation != 0).")

Pearson Correlation Coefficient: 0.0224
P-value: 0.2054
Fail to reject H0: No significant correlation (correlation = 0).


In [38]:
state_data

,YearEnd,LocationAbbr,State,DataSource,Topic,Question,DataValueUnit,DataValueType,DataValue,DataValueAlt,...,LocationID,TopicID,QuestionID,DataValueTypeID,StratificationCategoryID1,StratificationID1,Overall Public School Rank 2021,Higher Ed Quality 2021,School Safety Rank 2021,Ranking Level
0,2019,AK,Alaska,NVSS,Cardiovascular Disease,Diseases of the heart mortality among all peop...,"cases per 100,000",Age-adjusted Rate,159.70,159.70,...,2,CVD,CVD09,AGEADJRATE,SEX,SEXM,48,50,34,Bottom 10
1,2019,CA,California,BRFSS,Cardiovascular Disease,Taking medicine to control high blood pressure...,%,Age-adjusted Prevalence,47.20,47.20,...,6,CVD,CVD02,AGEADJPREV,RACE,MRC,37,38,32,Middle
2,2019,AL,Alabama,CMS Part A Claims Data,Cardiovascular Disease,Hospitalization for heart failure as principal...,"cases per 1,000",Crude Rate,71.43,71.43,...,1,CVD,CVD06,CRDRATE,RACE,AIAN,44,46,17,Bottom 10
3,2019,CA,California,BRFSS,Cardiovascular Disease,Taking medicine for high cholesterol among adults,%,Crude Prevalence,31.60,31.60,...,6,CVD,CVD04,CRDPREV,SEX,SEXM,37,38,32,Middle
4,2019,AZ,Arizona,NVSS,Cardiovascular Disease,Cerebrovascular disease (stroke) mortality amo...,"cases per 100,000",Crude Rate,26.10,26.10,...,4,CVD,CVD07,CRDRATE,RACE,BLK,49,50,31,Bottom 10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16675,2021,WY,Wyoming,BRFSS,Cardiovascular Disease,High cholesterol among adults who have been sc...,%,Crude Prevalence,32.80,32.80,...,56,CVD,CVD03,CRDPREV,AGE,AGE4564,19,18,29,Middle
16676,2021,WY,Wyoming,BRFSS,Cardiovascular Disease,High blood pressure among adults,%,Age-adjusted Prevalence,39.90,39.90,...,56,CVD,CVD01,AGEADJPREV,RACE,HIS,19,18,29,Middle
16677,2021,WY,Wyoming,BRFSS,Cardiovascular Disease,High blood pressure among adults,%,Crude Prevalence,32.20,32.20,...,56,CVD,CVD01,CRDPREV,RACE,MRC,19,18,29,Middle
16678,2021,WY,Wyoming,NVSS,Cardiovascular Disease,Coronary heart disease mortality among all peo...,"cases per 100,000",Crude Rate,50.70,50.70,...,56,CVD,CVD08,CRDRATE,RACE,HIS,19,18,29,Middle


In [34]:
states_data

,Overall Public School Rank 2022,Alcohol,Arthritis,Asthma,Cancer,Chronic Obstructive Pulmonary Disease,Cognitive Health and Caregiving,Diabetes,Disability,Health Status,Immunization,Mental Health,"Nutrition, Physical Activity, and Weight Status",Oral Health,Sleep,Social Determinants of Health,Tobacco
0,32,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
1,19,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
2,44,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
3,44,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
4,44,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29908,39,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
29909,5,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
29910,39,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
29911,8,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False


In [40]:
XKNN = states_data.drop('Overall Public School Rank 2021', axis=1)
yKNN = states_data['Overall Public School Rank 2021'].values
XKNN_train, XKNN_test, yKNN_train, yKNN_test = train_test_split(XKNN ,yKNN, test_size = 0.2, random_state = 42)
knn = KNeighborsClassifier(n_neighbors =7)
knn.fit(XKNN_train, yKNN_train)
yKNN_pred = knn.predict(XKNN_test)
print(confusion_matrix(yKNN_test, yKNN_pred))

[[ 0 82  0 ...  0  0  0]
 [ 0 73  0 ...  0  0  0]
 [ 0 53  0 ...  0  0  0]
 ...
 [ 0 94  0 ...  0  0  0]
 [ 0 62  0 ...  0  0  0]
 [ 0 73  0 ...  0  0  0]]


In [42]:
print(classification_report(yKNN_test, yKNN_pred))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00        82
           2       0.02      1.00      0.04        73
           3       0.00      0.00      0.00        53
           4       0.00      0.00      0.00        72
           5       0.00      0.00      0.00        41
           6       0.00      0.00      0.00        59
           7       0.00      0.00      0.00        75
           8       0.00      0.00      0.00        79
           9       0.00      0.00      0.00        60
          10       0.00      0.00      0.00        69
          11       0.00      0.00      0.00        68
          12       0.00      0.00      0.00        83
          13       0.00      0.00      0.00        73
          14       0.00      0.00      0.00        50
          15       0.00      0.00      0.00        61
          16       0.00      0.00      0.00        68
          17       0.00      0.00      0.00        71
          18       0.00    

/opt/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [44]:
f1_score(yKNN_test, yKNN_pred, average= 'weighted')

0.0009371792680258844